# Demo AMBIT query

- reads a file into dataframe
- retrieve chemical structure by identifier given as dataframe  column  via AMBIT API
- retrieve substance related information by querying substance by name given as dataframe  column  via AMBIT API

In [ ]:
import importlib
from pynanomapper import aa
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr
from pynanomapper import client_ambit
from pynanomapper import annotation
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def search_service_protected(url_ambit,apikey):
    return (url_ambit,apikey)

service_widget =widgets.Text(
       placeholder='',
       description="AMBIT URL",
       disabled=False,
       style={'description_width': 'initial'}
) 
apikey_widget=widgets.Text(
          placeholder='',
           description="API key",
           disabled=False,
           style={'description_width': 'initial'}
)    
service = interactive(search_service_protected,url_ambit=service_widget,apikey=apikey_widget)    
display(service)

In [ ]:
ambit_uri=service_widget.value
auth_object = aa.AmbitAPIKEYAuth()
auth_object.setKey(apikey_widget.value)

#getSubstanceComposition(http,ambit_url,substance_uuid):

In [ ]:
def input_options(file,query_column):
    return (file,query_column)

file_widget =widgets.Text(
       placeholder='',
       description="File path",
       disabled=False,
       style={'description_width': 'initial'}
) 
query_column_widget =widgets.Text(
       value='Substance Number',
       description="Column name",
       disabled=False,
       style={'description_width': 'initial'}
) 

interactive(input_options,file=file_widget,query_column=query_column_widget)    


In [ ]:
# read the file to iterate
df = pd.read_excel(file_widget.value) 


In [ ]:
from  rdkit  import  Chem
importlib.reload(client_ambit)

cli = client_ambit.AMBITQueryCompound(ambit_uri)

def create_smiles(molblock):
    m = Chem.MolFromMolBlock(molblock,strictParsing=False)
    if m is None:
        smi=None
    else:
        smi = Chem.MolToSmiles(m,isomericSmiles=True)
    return smi  


def query2smiles(q):
    
    if pd.isnull(q):
        return q
    else:
        try:
            r = cli.get(params={'search':q,'type':'auto'},media='chemical/x-mdl-sdfile',auth=auth_object)
            if r.status_code==200:
                return create_smiles(r.text)    
            else:
                return None
        except Exception as err:    
            print(err)
            return None
#query by value in the specified column name
smiles = df[query_column_widget.value].apply(query2smiles)
df['SMILES']=smiles
df.head()
df.to_excel(file_widget.value.replace(".xlsx","_SMILES.xlsx"))

In [ ]:
cli = client_ambit.AMBITSubstance(ambit_uri)
def query2substancetype(q):
    
    if pd.isnull(q):
        return q
    elif q=="-":
        return None
    elif q.strip()=="":
        return None    
    else:
        try:
            r = cli.get(params={'search':"^{}$".format(q),'type':'regexp'},media='application/json',auth=auth_object)
            if r.status_code==200:
                substance = cli.parse(r.json())   
                if len(substance)>0: 
                    return substance[0]['substanceType']
                else:
                    return None
                    
            else:
                return None
        except Exception as err:    
            print(err)
            return None
        
results = df["Substance Name"].apply(query2substancetype)                    
#query2substancetype("(1-methyl-1,2-ethanediyl)bis[oxy(methyl-2,1-ethanediyl)] diacrylate")        
#results

In [ ]:
df['substance_type']=results
df.head()
df.to_excel(file_widget.value.replace(".xlsx","_SMILES_substancetype.xlsx"))